In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from cvxopt import solvers
from cvxopt import matrix
from scipy.spatial.distance import cdist
import scipy as scip
import numpy.matlib

In [19]:
titanic = pd.read_csv('final_dataset_ready.csv')
dataset = titanic.copy()
X = np.asarray(dataset.drop('Survived', 1))
y = np.asarray(dataset['Survived'])
y[y==0] = -1

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [21]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [22]:
def getRBFKernelMatrix(X1,X2,s):
    
  # this is an NxD matrix, where N is number of items and D its dimensionalites
    #pairwise_sq_dists = squareform(pdist(X, 'sqeuclidean'))
    pairwise_sq_dists = cdist(X1, X2, 'sqeuclidean')
    K = scip.exp(-pairwise_sq_dists / s**2)
    return K

In [23]:
s = 0.5
C = 1
P = getRBFKernelMatrix(X_train,X_train,s)
N = X_train.shape[0]
q = -1*np.ones((N,1))
G = np.diagflat(-1*q)
G = np.vstack((G,-1*G))
h = np.vstack((C*np.ones((N,1)),np.zeros((N,1))))
h = np.squeeze(h)
q = np.squeeze(q)
A = y_train
b = np.array([0])

In [24]:
P = matrix(P,tc='d')
q = matrix(q,tc='d')
G = matrix(G,tc='d')
h = matrix(h,tc='d')
A = matrix(A,tc='d')
b = matrix(b,tc='d')
A = np.asarray(A)

if A.shape[0] > A.shape[1]:
    A = A.T    
A = matrix(A,tc='d')

In [25]:
a = solvers.qp(P, q, G, h, A, b)
a = np.asarray(a['x'])

     pcost       dcost       gap    pres   dres
 0: -1.1179e+02 -1.0467e+03  9e+02  1e-14  8e-16
 1: -1.1990e+02 -1.7747e+02  6e+01  2e-15  7e-16
 2: -1.2502e+02 -1.4014e+02  2e+01  2e-15  4e-16
 3: -1.2679e+02 -1.3017e+02  3e+00  2e-15  4e-16
 4: -1.2722e+02 -1.2811e+02  9e-01  4e-15  4e-16
 5: -1.2733e+02 -1.2759e+02  3e-01  2e-14  5e-16
 6: -1.2737e+02 -1.2742e+02  5e-02  1e-14  4e-16
 7: -1.2738e+02 -1.2739e+02  2e-02  9e-15  4e-16
 8: -1.2738e+02 -1.2738e+02  2e-03  2e-15  4e-16
 9: -1.2738e+02 -1.2738e+02  2e-04  1e-14  4e-16
10: -1.2738e+02 -1.2738e+02  3e-05  2e-15  4e-16
Optimal solution found.


In [26]:
idx1 = a>=1e-10
idx2 = a<C
idx = np.logical_and(idx1,idx2)

In [27]:
Xy = X_train.T.dot(np.diagflat(y_train)).T
w0 = y_train-np.squeeze(getRBFKernelMatrix(Xy,X_train,s).dot(a))
w0 =np.mean(w0[np.squeeze(idx)])
yPred = np.sign(getRBFKernelMatrix(X_test,Xy,s).dot(a)+w0)

In [28]:
print(round(accuracy_score(y_test, yPred)*100,2))

72.63
